# Model 1

In this document I describe a simulation of an economy that consists of the following agents:

1. B2B company
2. B2C company
3. Workers (emplyees of B2B and B2C companies)
4. Financial system
5. B2B market
6. B2C market

Their purpose will become evident in the following description of simulation steps.

In [13]:
from graphviz import Digraph
from graphviz import Digraph
from graphviz import Graph
import copy
import sympy
from sympy import symbols
from IPython.display import HTML, display
import tabulate

In [17]:
import Unit
import InventoryTx
import Inventory
import AccountingTx
import Location
import SimulationParameters
import State
import Resource
import Amount
import StateTransition

In [19]:
fs = Location.Location(Location.Location.FS)
fs.accounts.append(Account(Account.ACCOUNTS_RECEIVABLE))

b2bCompany = Location(Location.B2B_COMPANY)
b2bCompany.accounts.append(Account(Account.ACCOUNTS_PAYABLE))
b2bCompany.accounts.append(Account(Account.CASH))

b2cCompany = Location(Location.B2C_COMPANY)
b2cCompany.accounts.append(Account(Account.ACCOUNTS_PAYABLE))

marketB2B = Location(Location.B2B_MARKET)
marketB2C = Location(Location.B2C_MARKET)
b2bWorker = Location(Location.B2B_WORKER)
b2bWorker.accounts.append(Account(Account.CASH))
b2cWorker = Location(Location.B2C_WORKER)

NameError: name 'Inventory' is not defined

In [ ]:
initialState = State()
initialState.locations.append(fs)
initialState.locations.append(b2bCompany)
initialState.locations.append(marketB2B)
initialState.locations.append(b2cCompany)
initialState.locations.append(marketB2C)
initialState.locations.append(b2bWorker)
initialState.locations.append(b2cWorker)

In [ ]:
def acctView(transition, state):
    newState = transition.run(state)
    transition.acctView(newState)

In [ ]:
simSteps = []

In [ ]:
currentState = initialState

This is what the accounts look like in the initial state of simulation:

In [ ]:
currentState.acctView()

![Overview over simulation components](img/img04.png)

## Simulation Steps

### Step 1: Financial system generates money

![Step 1](img/img05.png)

In this step, the financial system (including private and the central bank) generates two amounts of money:

* `M1`: Money, which will be used by the B2B company to pay for everything it needs to produce its output.
* `M2`: Money for the B2C company.

In this simulation we ignore the fact that both companies need to buy inputs (raw materials, energy) in order to produce. `M1` and `M2` will be used by the companies to pay the wages of the workers (which must be paid in advance, i. e. before companies sold their products on the market).

In this step, the financial system generates the required amounts of money, unless it already has them. We assume that the financial system can generate (print) money however it wants. Money is not backed by any asset (like gold).

In [ ]:
class FinancialSystemGeneratesMoney(StateTransition):
    def run(self, oldState):        
        newState = copy.deepcopy(oldState)
        newState.step = oldState.step + 1
        newState.comment = "Financial system generates (adjusts) the amount of money"
        self.put(newState, Resource.B2B_LOAN, Location.FS)
        self.put(newState, Resource.B2C_LOAN, Location.FS)
        return newState


In [ ]:
currentState = FinancialSystemGeneratesMoney().run(currentState)

#### Accounting perspective

This step does not have an accounting representation because

* essentially, the financial system generates money out of thin air and
* normal accounting principles don't work well with such phenomena.

Therefore there are no accounting transactions in this step.

In [ ]:
# TODO: Can we remove simSteps?

simSteps.append(FinancialSystemGeneratesMoney())

### Step 2: Financial System lends money to companies

Financial System lends money `M1` and `M2` to B2B and B2C companies, respectively, at interest rate `I0`. Two money cubes (`M1` and `M2`) move from the financial system to the respective company (see red ellipses).

At the same time, each cubes gets an additional piece of information attached to it -- the interest rate `I0`, by which the debt will grow at the end of the cycle.

![Step 2: Financial System lends money to companies](img/img06.png)


In [ ]:
class FinancialSystemInjectsLoans(StateTransition):
    def run(self, oldState):
        newState = copy.deepcopy(oldState)
        simStep = oldState.step + 1
        newState.step = simStep
        newState.comment = "Financial system injects loans with a certain interest rate to the companies"
        self.move(newState, Resource.B2B_LOAN, Location.FS, Location.B2B_COMPANY)
        self.move(newState, Resource.B2C_LOAN, Location.FS, Location.B2C_COMPANY)
        
        # B2B_LOAN (start)
        self.put(newState, Resource.B2B_LOAN, Location.FS)
        tx = AccountingTx(Unit.CURRENCY, "M1", simStep, "FS's loan to B2B company")
        cAcct = self.findAccount(newState, Location.B2B_COMPANY, Account.ACCOUNTS_PAYABLE)
        dAcct = self.findAccount(newState, Location.FS, Account.ACCOUNTS_RECEIVABLE)
        self.acctTx(tx, cAcct, dAcct)
        # B2B_LOAN (end)
        
        # B2C_LOAN (start)
        self.put(newState, Resource.B2C_LOAN, Location.FS)
        tx = AccountingTx(Unit.CURRENCY, "M2", simStep, "FS's loan to B2C company")
        cAcct = self.findAccount(newState, Location.B2C_COMPANY, Account.ACCOUNTS_PAYABLE)
        dAcct = self.findAccount(newState, Location.FS, Account.ACCOUNTS_RECEIVABLE)
        self.acctTx(tx, cAcct, dAcct)
        # B2C_LOAN (end)
        
        return newState

In [ ]:
print(",\n".join(str(x) for x in currentState.locations))

In [ ]:
tempFs = next(x for x in currentState.locations if x.name == Location.FS)
print("Found: " + str(tempFs))
tempFs.objects

In [ ]:
currentState = FinancialSystemInjectsLoans().run(currentState)

#### Accounting perspective

This is what this step looks like from an accountant's point of view:

In [ ]:
currentState.acctView()

### Step 3: B2B company pays wages to its worker

In this step, the B2B company pays its worker the wages (from the `M1` money). We leave the cube at the B2B company to indicate the debt. At the same time, another money cube `M3` appears at the B2B worker.

![Step 3: B2B company pays wages to its worker](img/img07_step03.png)


In [ ]:
class B2BCompanyPaysEmployee(StateTransition):
    def run(self, oldState):
        newState = copy.deepcopy(oldState)
        simStep = oldState.step + 1
        newState.step = simStep
        newState.comment = "B2B Company Pays its Employees"
        b2bWorker = self.findByName(newState.locations, Location.B2B_WORKER)
        wages = Resource(Resource.WAGES_B2B)
        b2bWorker.objects.append(wages)
        # Accounting transaction (start)    
        tx = AccountingTx(Unit.CURRENCY, "M3", simStep, "Wages paid by the B2B company to B2B worker")
        cAcct = self.findAccount(newState, Location.B2B_COMPANY, Account.CASH)
        dAcct = self.findAccount(newState, Location.B2B_WORKER, Account.CASH)
        self.acctTx(tx, cAcct, dAcct)        
        # Accounting transaction (end)
        return newState


In [ ]:
currentState = B2BCompanyPaysEmployee().run(currentState)

#### Accounting perspective

In [ ]:
currentState.acctView()

In this step the wages `M3` flow from the B2B company to B2B worker. B2B Company's cash account is credited with `M3`, B2B worker's cash account is credited with the same amount.

### Step 4: The B2B company produces its output

The B2B product is denoted `R1`. `R` means *resource*.

![Step 4: The B2B company produces its output](img/img08_step04.png)


In [ ]:
class B2BCompanyProduces(StateTransition):
    def run(self, oldState):
        newState = copy.deepcopy(oldState)
        simStep = oldState.step + 1
        newState.step = simStep
        newState.comment = "B2B Company Produces"
        b2bCompany = self.findByName(newState.locations, Location.B2B_COMPANY)
        b2bProduct = Resource(Resource.B2B_PRODUCT)
        # name, unit, quantity
        cycleDurationInYears = symbols("cycleDurationInYears")
        numberOfB2BWorkers = symbols("numberOfB2BWorkers")
        reproductionValueExpr = cycleDurationInYears * numberOfB2BWorkers
        reproductionValue = Amount(Amount.REPRODUCTION_VALUE,
                                   Unit.LABOR_YEAR, 
                                   reproductionValueExpr)
        b2bProduct.addAmount(reproductionValue)
        b2bProductProduced = symbols("b2bProductProduced")
        b2bQuantityExpr = b2bProductProduced
        b2bQuantity = Amount(Amount.PRODUCT_QUANTITY, Unit.PIECE, b2bQuantityExpr)
        b2bProduct.amounts.append(b2bQuantity)
        b2bCompany.objects.append(b2bProduct)
        
        # Inventory transaction
        inventoryTx = InventoryTx(Resource.B2B_PRODUCT, Unit.LABOR_YEAR, 1.0, simStep, "B2B company produces")
        b2bCompany.inventory.addTx(inventoryTx)
        return newState


In [ ]:
currentState = B2BCompanyProduces().run(currentState)

#### Accounting perspective

In this step there are no monetary movements between agents. However, B2B company produces something that did not exist before (B2B product) and this needs to be noted somehow.

B2B product appears in the list of inventory transactions of B2B company.

In [ ]:
currentState.acctView()

In [ ]:
simSteps.append(B2BCompanyProduces())

### Step 5: B2B market setup

In this step

* the B2B company puts its output to the B2B market and
* the B2C company decides, how much money (out of the `M2` loan) it is willing to spend on B2B product.

This defines the limitations of the amount of B2B product that can change hands in this market at this time:

* The B2B company cannot sell more product than it has.
* The B2C company cannot spend more money than `M4`.

![Step 5: B2B market setup](img/img09_step05.png)

#### Accounting perspective

TODO

In [ ]:
# TODO, continue here: Step 5, B2B market setup. Make provisions for modeling 
# several cases:
# 1) perfect market clearing
# 2) excess demand
# 3) excess supply
#
# Maybe create a function that models the logic behind the decisions of
# a. B2B company of how much to produce and
# b. B2C company of how much to buy.

From the accounting perspective, nothing happens in this step because it is used for better visualization only.

In [5]:
class B2BCompanySellsProductToB2CCompany1(StateTransition):
    def run(self, oldState):
        newState = copy.deepcopy(oldState)
        newState.step = oldState.step + 1
        newState.comment = "B2B company sells product to B2C company (1)"
        self.move(newState, Resource.B2B_PRODUCT, Location.B2B_COMPANY, Location.MARKET_B2B)
        self.put(newState, Resource.MONEY_FOR_B2B_PRODUCT, Location.MARKET_B2B)
        return newState


TypeError: module.__init__() takes at most 2 arguments (3 given)

In [ ]:
simSteps.append(B2BCompanySellsProductToB2CCompany1())

### Step 6: B2B company sells its product to B2C company

The market decides

* how much product the B2C company buys and
* how much money the B2B company gets for it.

Thereafter, 

* the `R1` cube (B2B product) moves to the B2C company and 
* the `M4` (the price for `R1` as determined by the market) moves to the B2B company.

![Step 6: B2B company sells its product to B2C company](img/img10_step06.png)


#### Accounting perspective

TODO

In [ ]:
class B2BCompanySellsProductToB2CCompany2(StateTransition):
    def run(self, oldState):
        newState = copy.deepcopy(oldState)
        newState.step = oldState.step + 1
        newState.comment = "B2B company sells product to B2C company (2)"
        self.move(newState, Resource.B2B_PRODUCT, Location.MARKET_B2B, Location.B2C_COMPANY)
        self.move(newState, Resource.MONEY_FOR_B2B_PRODUCT, Location.MARKET_B2B, Location.B2B_COMPANY)        
        return newState


In [ ]:
simSteps.append(B2BCompanySellsProductToB2CCompany2())

### Step 7: B2C company pays its worker wages

In this step the B2C company takes off a piece from the `M2` cube and gives it to the B2C worker.

![Step 7: B2C company pays its worker wages](img/img11_step07.png)

#### Accounting perspective

TODO

In [ ]:
class B2CCompanyPaysEmployee(StateTransition):
    def run(self, oldState):
        newState = copy.deepcopy(oldState)
        newState.step = oldState.step + 1
        newState.comment = "B2C Company Pays its Employees"
        b2cWorker = self.findByName(newState.locations, Location.PERSON_2)
        wages = Resource(Resource.WAGES_B2C)
        b2cWorker.objects.append(wages)
        return newState


In [ ]:
simSteps.append(B2CCompanyPaysEmployee())

### Step 8: B2C company produces

B2C company produces B2C product `R2` using B2C worker's labor from the B2B product `R1`.

![Step 8: B2C company produces](img/img12_step08a.png)

#### Accounting perspective

TODO

In [ ]:
class B2CCompanyProduces(StateTransition):
    def run(self, oldState):
        newState = copy.deepcopy(oldState)
        newState.step = oldState.step + 1
        newState.comment = "B2C Company Produces"
        b2cCompany = self.findByName(newState.locations, Location.B2C_COMPANY)
        b2cProduct = Resource(Resource.B2C_PRODUCT)
        b2cCompany.objects.append(b2cProduct)
        self.removeFromLoc(newState, Location.B2C_COMPANY, Resource.B2B_PRODUCT)
        return newState

In [ ]:
simSteps.append(B2CCompanyProduces())

### Step 9: B2C market setup

In this step

* B2C company puts the B2C product on the market and
* the customers (B2B and B2C workers) put the max. amount of money (`M6`) they are able to spend on B2C product.

![Step 9: B2C market setup](img/img13_step09.png)

#### Accounting perspective

TODO

In [ ]:
class ConsumersBuyB2CProducts1(StateTransition):
    def run(self, oldState):
        newState = copy.deepcopy(oldState)
        newState.step = oldState.step + 1
        newState.comment = "Consumers buy B2C products (part 1)"
        # Insert your transformations here
        self.move(newState, Resource.B2C_PRODUCT, Location.B2C_COMPANY, Location.MARKET_B2C)
        self.move(newState, Resource.WAGES_B2C, Location.PERSON_2, Location.MARKET_B2C)
        self.move(newState, Resource.WAGES_B2B, Location.PERSON_1, Location.MARKET_B2C)
        
        return newState


In [ ]:
simSteps.append(ConsumersBuyB2CProducts1())

### Step 10: B2C sells its product to customers

In this step

* the B2C product (`R2`) travels from B2C company to the consumers (B2B and B2C workers) and
* the money paid for it by the consumers (`M6`) travels to the B2C company.

![Step 10: B2C sells its product to customers](img/img14_step10.png)

#### Accounting perspective

TODO

In [ ]:
class ConsumersBuyB2CProducts2(StateTransition):
    def run(self, oldState):
        newState = copy.deepcopy(oldState)
        newState.step = oldState.step + 1
        newState.comment = "Consumers buy B2C products (part 2)"
        # Insert your transformations here
        self.removeFromLoc(newState, Location.MARKET_B2C, Resource.WAGES_B2C)
        self.removeFromLoc(newState, Location.MARKET_B2C, Resource.WAGES_B2B)
        self.put(newState, Resource.MONEY_FOR_B2C_PRODUCT, Location.B2C_COMPANY)
        self.removeFromLoc(newState, Location.MARKET_B2C, Resource.B2C_PRODUCT)
        self.put(newState, Resource.B2C_PRODUCT, Location.PERSON_1)
        self.put(newState, Resource.B2C_PRODUCT, Location.PERSON_2)
        return newState


In [ ]:
simSteps.append(ConsumersBuyB2CProducts2())

### Step 11: Interests grow

At the beginning of the simulation the financial system lends both companies money (`M1` and `M2`) at interest ratet `I0`. At the end of the year the debt of both companies to the financial system grows -- this additional debt is depicted as the `M7` and `M8` cubes.

![Step 11: Interest rate grows](img/img15_step11.png)

#### Accounting perspective

TODO

### Step 12: B2B and B2C companies pay pack their loans with interest

In this step both companies pay back

* the principals (`M1` and `M2`) and
* the interests (`M7` and `M8`)

to the financial system.

![Step 12: B2B and B2C companies pay pack their loans with interest](img/img16_step12.png)

#### Accounting perspective

TODO

## Simulation parameters

* `M1` (unit: money): Amount of money lent by the financial system to the B2B company
* `M2` (unit: money): Amount of money lent by the financial system to the B2C company
* `M3` (unit: money): Wages of the B2B worker per year
* `I0` (percent): Interest rate for `M1` and `M2` loans.

## Scenarios

Now we can run several scenarios, such as the following:

1. Zero interest rate, both markets clear
2. Zero interest rate, excess supply on the B2C market
3. Zero interest rate, excess demand on the B2C market
4. 10 % interest rate, both markets clear
5. 20 % interest rate, both markets clear
6. 30 % interest rate, both markets clear
7. 10 % interest rate, excess supply on the B2C market
8. 10 % interest rate, excess supply on the B2C market
9. 10 % interest rate, excess supply on the B2C market

Clarifications regarding terms are necessary.

## Market clearing

I say that a market **clears**, when, as a result of exchange between producer and consumer, the producer gets such an amount of money that

* the producer can pay for the costs of producing the good (wages, raw materials such as `R1`) and
* the loan and interest

with the money received from the buyers of its product. The amount received from the buyers can be higher, but not lower than this (if it's lower, then the market does not clear).

A market clears, when the producer of the good created

* exactly the product that the customers (buyers) want in
* exactly the quantity they want to buy.

## Excess supply

If a producer of a good tries to sell

* the wrong product (i. e. something that the customers don't want) or
* too much of the product,

then I call this situation **excess supply**.

## Excess demand

That's when a producer tries to sell

* the right product (something the buyers want) in
* too small quantity.